In [ ]:
from App.GUI import GUI
from App.DemoController import DemoController
import os

print("🩸👁️ Retinal Vessel Segmentation Application")
print("=" * 45)

# Initialize demo controller (automatically loads demo model)
controller = DemoController(auto_load_demo=True)

print(f"\n📋 Available segmentation methods: {controller.get_available_methods()}")

if 'deep_learning' in controller.get_available_methods():
    model_info = controller.get_model_info()
    print(f"\n🤖 Deep Learning Model Info:")
    print(f"   Model Type: {model_info['model_type']}")
    print(f"   Parameters: {model_info['parameters']:,}")
    print(f"   Device: {model_info['device']}")
    print(f"   Target Size: {model_info['target_size']}")
else:
    print("\n⚠️ Deep learning model not available, using filter-based segmentation only")

print("\n🖥️ Initializing GUI...")

# Initialize and launch GUI
gui = GUI(controller)
gui.init()

print("✅ GUI ready! Use the interface above to:")
print("   1. Upload retinal images")
print("   2. Select segmentation method(s):")
print("      • Prosty Model = Filter-based segmentation")
print("      • DL Model = Deep learning segmentation") 
print("   3. Click Start to process")
print("\n🎯 Ready for vessel segmentation!")

[DLUnet v2.0.0] Retinal Vessel Segmentation Package Loaded
Available interfaces:
  • VesselSegmentationPipeline - Complete training/inference pipeline
  • VesselSegmentationController - Simple controller interface
  • Core components for custom implementations


# Wykrywanie naczyń dna siatkówki oka

## Opis

Przygotowanie aplikacji, która dla zadanego obrazu wejściowego przedstawiającego dno siatkówki oka automatycznie wykrywa naczynia krwionośne. Z formalnego punktu widzenia dla każdego piksela wykorzystany algorytm musi stwierdzić, czy ten piksel stanowi naczynie krwionośne, czy nie (klasyfikacja binarna).

## Wymagania obowiązkowe

Algorytm w podstawowej wersji powinien wykorzystywać techniki przetwarzania obrazu do detekcji naczyń krwionośnych. W ramach takiego procesu przetwarzania można wyróżnić 3 główne elementy:

1. **Wstępne przetworzenie obrazu**  
   Wejściowy obraz może być zaszumiony/zbyt ciemny/jasny. Można tutaj wykorzystać takie techniki jak: rozmycie, wyostrzenie, normalizacja histogramu kolorów itp.

2. **Właściwe przetworzenie obrazu**  
   W celu wyodrębnienia naczyń krwionośnych można zastosować różne techniki wykrywania krawędzi (np. filtr Frangi’ego).

3. **Końcowe przetwarzanie obrazu**  
   Przetwarzanie uzyskanego obrazu w celu poprawy skuteczności wykrywania naczyń poprzez “naprawę” błędów z poprzednich kroków.

Wynik należy wizualizować, np. zamalowując wyróżniającym się kolorem piksele zaklasyfikowane jako naczynie krwionośne. Najlepiej wygenerować binarną maskę odpowiedzi algorytmu, która zostanie potem wykorzystana do analizy statystycznej (porównanie z maską ekspercką z ręcznie zaznaczonymi naczyniami).

Ważnym elementem oceny jest skuteczność algorytmu. W tym celu należy dokonać podstawowej analizy statystycznej jakości działania algorytmu. Działanie programu należy przetestować na minimum 5 obrazach. Podczas testów należy wyznaczyć:

- macierze pomyłek,
- trafność (accuracy),
- czułość (sensitivity),
- swoistość (specificity).

Przy wyznaczaniu czułości i swoistości należy założyć, że naczynie to klasa pozytywna, a tło — negatywna. Ponieważ mamy do czynienia z niezrównoważonym rozkładem klas, należy dodatkowo wykorzystać miary dla danych niezrównoważonych (np. średnią arytmetyczną lub geometryczną czułości i swoistości).

## Wymagania na 4.0

Po wstępnym przetworzeniu obrazu należy podzielić go na wycinki (np. 5x5 px) i dla każdej z nich dokonać ekstrakcji cech z obrazu: np. wariancja kolorów, momenty centralne, momenty Hu itp. Wartości te wraz z informacją pochodzącą z maski eksperckiej (decyzja dla środkowego piksela wycinka) stanowić będą zbiór danych wykorzystany do budowy wybranego klasyfikatora, prostszego niż głęboka sieć neuronowa (np. kNN, drzewo lub las decyzyjny, SVM). Należy skorzystać z gotowej implementacji klasyfikatora (np. w bibliotece scikit-learn).

Z uwagi na ograniczenia pamięciowe konieczne może być ograniczenie rozmiaru zbioru uczącego poprzez losowy wybór punktów (możliwość zastosowania undersamplingu do zrównoważenia rozkładu klas w zbiorze uczącym).

Zdolności predykcyjne tak opracowanego klasyfikatora należy wstępnie zweryfikować na niezależnym zbiorze testowym hold-out (np. pochodzącym z innej części obrazu lub z innego obrazu).

Gotowy klasyfikator powinien zostać osadzony w aplikacji, a jego działanie powinno zostać zwizualizowane i przetestowane w taki sam sposób, jak działanie technik przetwarzania obrazu z wymagań podstawowych.

## Wymagania na 5.0

Jako model decyzyjny należy wykorzystać głęboką sieć neuronową. W zależności od wybranego rodzaju sieci, może zostać ona nauczona na wycinkach obrazu (podobnie jak w przypadku wymagań na 4.0), jak i na całych obrazach (np. w przypadku sieci UNet). Należy skorzystać z gotowej implementacji sieci (np. w bibliotece Keras, PyTorch lub TensorFlow).

Zdolności predykcyjne nauczonej sieci neuronowej powinny być wstępnie zweryfikowane na zbiorze testowym hold-out.

Nauczona sieć powinna zostać osadzona w aplikacji i tam dodatkowo przetestowana zgodnie z wymaganiami obowiązkowymi.

## Uwaga

W projekcie należy skorzystać z jednej z dostępnych baz danych z obrazami (patrz linki poniżej) — ta sama baza powinna być stosowana we wszystkich krokach projektu.

## Linki

- Baza obrazów HRF: https://www5.cs.fau.de/research/data/fundus-images/  
  (proszę się nie przejmować komunikatem “website deprecated and outdated”).

- Baza obrazów STARE: http://cecas.clemson.edu/~ahoover/stare/probing/  
- Baza obrazów CHASE: https://blogs.kingston.ac.uk/retinal/chasedb1/  
  (przeglądanie zdjęć nie działa, można jednak pobrać całe archiwum)

- Biblioteka scikit-learn: https://scikit-learn.org/stable/  
- Biblioteka imbalanced-learn: https://imbalanced-learn.org/stable/  

- P. Liskowski, K. Krawiec: *Segmenting Retinal Blood Vessels With Deep Neural Networks*: https://ieeexplore.ieee.org/document/7440871  

- Implementacja sieci UNet z wykorzystaniem biblioteki PyTorch:  
  - https://pyimagesearch.com/2023/11/06/image-segmentation-with-u-net-in-pytorch-the-grand-finale-of-the-autoencoder-series/  
  - https://pyimagesearch.com/2021/11/08/u-net-training-image-segmentation-models-in-pytorch/  

- MONAI: Medical Open Network for AI: https://monai.io/  

- Przykładowa implementacja sieci UNet z wykorzystaniem biblioteki Keras:  
  - https://github.com/zhixuhao/unet  
  - https://github.com/karolzak/keras-unet  
  - https://github.com/qubvel/segmentation_models  

- Segmentacja obrazu z wykorzystaniem TensorFlow: https://www.tensorflow.org/tutorials/images/segmentation
